## Update Monitoring Data in SDE

#### This notebook converts a csv (with latitude and longitude WGS84 coordinates) into a temporary feature class, projects it to NAD83 UTM Zone10N, and then updates the corresponding SDE feature class with the new records.

#### Questions? 
#### Ask Mason Bindl, mbindl@trpa.org

##### Last updated 06/19/18

#### Update BikePed_Monitoring SDE data

In [1]:
# import modules
import arcpy, os, csv
from datetime import datetime
# start timer
startTimer = datetime.now()
print("Imported arcpy, os, csv, and datatime modules.\n")

## Set the Local Variables
#------------------------------------------------------------------------------------------------------#

# Change this to the path of your CSV file
incsv = r"C:\GIS\Dev\bike_ped_counter-2018-04-02-10-06-41.csv" 

# set base file (SDE Connection) for vector SDE
sdeBase = r"F:/GIS/GIS_DATA/Vector.sde/"

# Change this to the path of your output FC
outfc = sdeBase + "/sde.SDE.Transportation/sde.SDE.BikePed_Counter"

# set fields for field mapping variable
fields = ['month_day_year', 'counter_name', 'longitude', 'latitude', 'month_of_year', 'season_of_year', 
          'count_of_bike_ped', 'counter_category', 'SHAPE@XY']

#--------------------------------------------------------------------------------------------------------#
## Leave the rest alone
try:
    # unregister the sde feature class
    arcpy.UnregisterAsVersioned_management(outfc,"NO_KEEP_EDIT","COMPRESS_DEFAULT")

    # set overwrite files envrionment setting to True
    arcpy.env.overwriteOutput = True

    # in memory feature class to use before changing projecting to tempfc_project and writing to outfc
    wk_memory = "in_memory" + "\\"
    tempfc = wk_memory + "tempfc"

    # Create a temporary feature class on your local drive.
    # The project tool has to create a real feature class as it's output. This cannot be an in memory fc.
    tempfc_project = "C:/GIS/Dev/Dev.gdb/tempfc_project"

    # delete temporary feature classes if they still exist
    if arcpy.Exists(tempfc): 
        arcpy.Delete_management(tempfc)
        print ("\nDeleted tempfc\n")
    if arcpy.Exists(tempfc_project): 
        arcpy.Delete_management(tempfc_project)
        print ("\nDeleted tempfc_project\n")

    # Create the spatial reference object for WGS84. The bike ped data csv is WGS84. 
    spatialref = arcpy.SpatialReference(4326)
    # Create the spatial reference object for NAD83 UTM Zone 10N. All SDE data is NAD83 UTM Zone 10N
    outSpatialref = arcpy.SpatialReference(26910)

    # Create the temp feature class if needed
    if not arcpy.Exists(tempfc): 
        arcpy.CreateFeatureclass_management(os.path.dirname(tempfc), os.path.basename(tempfc), "POINT", None, None, None, spatialref)

    # Register the dialect for native CSV syntax in Microsoft Excel
    csv.register_dialect("xls", delimiter=",", lineterminator="\n") 
    f = open(incsv, "r") 
    reader = csv.reader(f, dialect = "xls")

    # reads the first line into a list of field names
    headers = next(reader) 

    # Add all fields. Default is TEXT field.
    for header in headers[0:]: 
        arcpy.AddField_management(tempfc, header, "TEXT")

    # Insert csv data into temporary feature class
    with arcpy.da.InsertCursor(tempfc, (['SHAPE@XY'] + headers[0:])) as cursor: 
        count = 0
        for row in reader: # loops through all the rows in the csv reader object
            Ycoord = row[0] # Make sure index value is set to field for 'Latitude'. 0 is the first field etc..
            Xcoord = row[1] # Make sure index value is set to field for 'Longitude'. 0 is the first field etc..
            newrow = [(float(Xcoord), float(Ycoord))] + row[0:]
            cursor.insertRow(newrow) # Insert point in FC for each row in CSV
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into temporary feature class".format(count))

    # project temp feature class from WGS84 to NAD83 UTM Zone 10N
    arcpy.Project_management(tempfc, tempfc_project, outSpatialref)
    print ("\nProjected Temporary Feature Class to NAD 83 UTM Zone 10N\n")

    # List all field names except the OID field
    dsc = arcpy.Describe(tempfc_project)
    fields = dsc.fields
    fieldnames = [field.name for field in fields if field.name != dsc.OIDFieldName]

    # check that field names are in outfc
    for f in fieldnames: 
        print("Field Exists: ", f, f in [field.name for field in arcpy.ListFields(tempfc_project)]) 

    # print new line
    print("\n")

    # change 'NA' values to NULL
    fieldstoChange = ["count_of_bike_ped", "month_day_year", "month_of_year"]
    with arcpy.da.UpdateCursor(tempfc_project, fieldstoChange) as uCursor:
        count = 0
        for row in uCursor:
            if (row[0] =='NA'):
                row[0] = None
            if (row[1] =='NA'):
                row[1] = None
            if (row[2] =='NA'):
                row[2] = None
            uCursor.updateRow(row)
            count += 1
            if count % 1000 == 0:
                print("Changing record {0} from 'NA' to 'NULL' in temporary feature class".format(count))

    # deletes all rows from the SDE feature class
    arcpy.TruncateTable_management(outfc)
    print ("\nDeleted all records in: {}\n".format(outfc))

    # insert rows from Temporary feature class to SDE feature class
    with arcpy.da.InsertCursor(outfc, field_names = fieldnames) as oCursor:
        count = 0
        with arcpy.da.SearchCursor(tempfc_project, field_names = fieldnames) as iCursor:
            for row in iCursor:
                oCursor.insertRow(row)
                count += 1
                if count % 1000 == 0:
                    print("Inserting record {0} into SDE feature class".format(count))

    # delete all temporary feature classes
    arcpy.Delete_management(tempfc)
    arcpy.Delete_management(tempfc_project)

    # register SDE feature class as versioned
    arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")

    # confirm feature class was created
    print("\nUpdated " + outfc)

    # report how long it took to run the script
    endTimer = datetime.now() - startTimer
    print ("\nTime it took to run this script: {}".format(endTimer))

except :
    
    print ("Data is locked...registering {0} feature dataset as versioned...".format(fdata))
    arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
    sys.exit()

except ValueError as err:
    
    print("Need ArcInfo license")
    arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
    sys.exit()
    


Imported arcpy, os, csv, and datatime modules.



SyntaxError: default 'except:' must be last (<ipython-input-1-40a7f44d8ce4>, line 140)

### Update INRIX data

In [6]:
import arcpy
from datetime import datetime
# start timer
startTimer = datetime.now()
print("Imported arcpy and datatime modules.\n")

infc = (r"F:\Research and Analysis\LT Info Monitoring Data\live_data\live_data.gdb\INRIX_Congestion")

# set base file (SDE Connection) for vector SDE
sdeBase = r"F:/GIS/GIS_DATA/Vector.sde/"

# Change this to the path of your output FC
outfc = sdeBase + "sde.SDE.LTINFO_Monitoring/sde.SDE.INRIX_Congestion"

# feature dataset to unversion and register as version
fdata = sdeBase + "/sde.SDE.LTINFO_Monitoring"

# unregister the sde feature class
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")

 # set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True
    
# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

fields = ['tmc', 'CongestionIndex', 'id', 'date']
# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fields) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(infc, fields) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 10000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))
    
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")
                    
# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))   

Imported arcpy and datatime modules.


Deleted all records in: F:/GIS/GIS_DATA/Vector.sde/sde.SDE.LTINFO_Monitoring/sde.SDE.INRIX_Congestion



RuntimeError: The requested operation is invalid on a closed state [sde.SDE.INRIX_Congestion]

### Reference
#### http://pro.arcgis.com/en/pro-app/tool-reference/data-management/truncatetable.htm
#### http://pro.arcgis.com/en/pro-app/tool-reference/data-management/project.htm
#### http://pro.arcgis.com/en/pro-app/arcpy/data-access/insertcursor-class.htm
#### http://pro.arcgis.com/en/pro-app/arcpy/data-access/searchcursor-class.htm
#### http://pro.arcgis.com/en/pro-app/arcpy/data-access/updatecursor-class.htm
#### https://docs.python.org/3/library/csv.html
#### https://gis.stackexchange.com/questions/90372/using-arcpy-da-insertcursor-to-insert-entire-row-that-is-fetched-from-search-cur
#### https://gis.stackexchange.com/questions/160002/event-layer-to-feature-class-fastest-method/160098
#### http://pro.arcgis.com/en/pro-app/tool-reference/data-management/unregister-as-versioned.htm
#### http://pro.arcgis.com/en/pro-app/tool-reference/data-management/register-as-versioned.htm
#### http://pro.arcgis.com/en/pro-app/arcpy/geoprocessing_and_python/accessing-licenses-and-extensions-in-python.htm